
# filesystem

a notebook full of filesystem utilities




## organize downloads




### organize downloads?

organize downloads folder using AI?


In [ ]:

//let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Default')







categorize home directory?


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')

const PROJECT_PATH = path.join(path.dirname(__dirname), '.categories.json')
const PROFILE_HOME = path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE)

let existingCategories = []
let mtime = ''
if (fs.existsSync(PROJECT_PATH)) {
  existingCategories = JSON.parse(fs.readFileSync(PROJECT_PATH))
  mtime = fs.statSync(PROJECT_PATH).mtime.toISOString()
}

async function categorizeHome() {
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Gemma')

  let basepath = PROFILE_HOME

  let projects = fs.readdirSync(basepath)
    .filter(dir => dir[0] != '.'
      && fs.statSync(path.join(basepath, dir)).isDirectory()
      && fs.existsSync(path.join(basepath, dir, '.git'))
      && fs.statSync(path.join(basepath, dir, '.git')).isDirectory()
      && !dir.includes('porn'))

  let q1 = 'Here\'s a list of existing projects and modified times:\n'
    + projects.map(dir => dir + ' - ' + fs.statSync(path.join(basepath, dir)).mtime.toISOString()).join('\n')
    + (existingCategories.length == 0 ? '' : ('\nHere is the list of existing categories modified on '
      + mtime + ' :\n' + existingCategories.join('\n')))
    + '\nPlease make an awesome list of categories, keep the category name simple, like one or two words.'
    + '\nAdd any new categories that seem necessary to accomodate any new projects.'
    + '\nOnly respond with the flat new line delimited list of categories and nothing else.'
  console.log('User:', q1)
  let a1 = await promptModel(q1)
  console.log('AI:', a1)

  let categories = a1.trim().split(/\s*\n\s*|,\s*|\s*- |\s*\*+\s*/gi)

  fs.writeFileSync(PROJECT_PATH, JSON.stringify(existingCategories.concat(categories), null, 4))
  // TODO: cache categories
  return categories

}

module.exports = categorizeHome

